In [13]:
import numpy as np
import pandas as pd
my_points = [[0.4005, 0.5306], 
             [0.2148, 0.3854],
             [0.3457, 0.3156],
             [0.2652, 0.1875],
             [0.0789, 0.4139],
             [0.4548, 0.3022]]
n_iter = 5
damping = 0.8

In [14]:
def sim (x, y):
    return -1*((x[0]-y[0])**2 + (x[1]-y[1])**2)

In [15]:
def solve_res(a, s, j):
    sij = s[j]
    s_needed = np.array(s[:j]+s[j+1:])
    a_needed = np.array(a[:j]+a[j+1:])
    return sij - max(s_needed + a_needed)

def respon(av_l,sim_l):
    l = []
    for i in range(len(my_points)):
        row=[]
        for j in range(len(my_points)):  
            responsibility= solve_res(av_l[i],sim_l[i], j)
            row.append(responsibility)
        l.append(row)
    return l

In [16]:
def solve_self_av(resp_col, i):
    out = 0
    for x in range(6):
        if(x==i):
            continue
        out= out+ max(0, resp_col[x])
    return out
def available(res_matrix, res_list):
    l = []
    for i in range(len(my_points)):
        row=[]
        for j in range(len(my_points)):
            if (i==j):
                self_avail= solve_self_av(resp_matrix['P'+str(j+1)].tolist(), i)
                row.append(self_avail)
            else:
                s_avail= solve_self_av(resp_matrix['P'+str(j+1)].tolist(), i)
                avail= min(0, resp_list[j][j]+s_avail)
                row.append(avail)
        l.append(row)
    return l

In [17]:
def update_AR(R,A,R_old,A_old):
    for i in range(6):
        R_old[i] = [x * damping for x in R_old[i]]
        R[i]=[x * (1-damping) for x in R[i]]
        R[i] = [a + b for a, b in zip(R[i], R_old[i])]
        A_old[i] = [x * damping for x in A_old[i]]
        A[i]=[x * (1-damping) for x in A[i]]
        A[i] = [a + b for a, b in zip(A[i], A_old[i])]
    return R, A

In [18]:
l = []
for point in my_points:
    row=[]
    for other in my_points:
        if (point==other):
            row.append(0)
            continue
        row.append(sim(point, other))
    l.append(row)

In [19]:
points=['P1', 'P2', 'P3', 'P4', 'P5', 'P6']
sim_matrix= pd.DataFrame(data=l, index=points, columns=points)
minimum = min(sim_matrix.min())
sim_matrix.replace(0,minimum,inplace=True)
sim_list = sim_matrix.values.tolist()
sim_matrix

,P1,P2,P3,P4,P5,P6
P1,-0.153778,-0.055568,-0.049228,-0.136024,-0.117045,-0.055115
P2,-0.055568,-0.153778,-0.022007,-0.041705,-0.019281,-0.064522
P3,-0.049228,-0.022007,-0.153778,-0.022890,-0.080845,-0.012082
P4,-0.136024,-0.041705,-0.022890,-0.153778,-0.085965,-0.049104
P5,-0.117045,-0.019281,-0.080845,-0.085965,-0.153778,-0.153778
P6,-0.055115,-0.064522,-0.012082,-0.049104,-0.153778,-0.153778


In [20]:
resp_list=[[0,0,0,0,0,0]] * 6
avail_list=[[0,0,0,0,0,0]] * 6
for i in range(5):
    old_R = resp_list
    old_A = avail_list
    resp_list=respon(avail_list,sim_list)
    resp_matrix=pd.DataFrame(data=resp_list, index=points, columns=points)
    avail_list = available(resp_matrix, resp_list)
    #if i!=0:
     #   resp_list, avail_list = update_AR(resp_list,avail_list,old_R,old_A)

In [21]:
resp_matrix=pd.DataFrame(data=resp_list, index=points, columns=points)
resp_matrix

,P1,P2,P3,P4,P5,P6
P1,-0.104550,-0.239968,-0.233629,-0.320425,-0.301446,-0.239516
P2,-0.467596,-0.134497,-0.434035,-0.453733,-0.431310,-0.476551
P3,-0.467059,-0.439837,-0.141695,-0.440720,-0.498676,-0.429913
P4,-0.397717,-0.303398,-0.284583,-0.130888,-0.347658,-0.310798
P5,-0.257512,-0.159748,-0.221312,-0.226431,-0.134497,-0.294244
P6,-0.317894,-0.327301,-0.274861,-0.311883,-0.416557,-0.141695


In [22]:
avail_matrix=pd.DataFrame(data=avail_list, index=points, columns=points)
avail_matrix

,P1,P2,P3,P4,P5,P6
P1,0.00000,-0.134497,-0.141695,-0.130888,-0.134497,-0.141695
P2,-0.10455,0.000000,-0.141695,-0.130888,-0.134497,-0.141695
P3,-0.10455,-0.134497,0.000000,-0.130888,-0.134497,-0.141695
P4,-0.10455,-0.134497,-0.141695,0.000000,-0.134497,-0.141695
P5,-0.10455,-0.134497,-0.141695,-0.130888,0.000000,-0.141695
P6,-0.10455,-0.134497,-0.141695,-0.130888,-0.134497,0.000000


In [11]:
C=resp_matrix+avail_matrix
C

,P1,P2,P3,P4,P5,P6
P1,-0.037038,-0.001253,0.000860,-0.119816,-0.110656,-0.042500
P2,-0.095423,0.073527,-0.039537,-0.093115,-0.080518,-0.107196
P3,-0.093458,-0.041028,0.068845,-0.078675,-0.146447,-0.059131
P4,-0.128713,-0.008364,0.008225,-0.039124,-0.101853,-0.052635
P5,-0.098053,0.032687,-0.039527,-0.082873,-0.063729,-0.137604
P6,-0.063696,-0.035872,0.016536,-0.069200,-0.171709,-0.023884


In [12]:
labels_our = [np.argmax(C.iloc[i]) for i in range(6)]
for i in range (6):
    print("P"+str(i+1),labels_our[i])

P1 2
P2 1
P3 2
P4 2
P5 1
P6 2
